In [1]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential,Model,initializers,layers,Input
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [19]:
import tensorflow as tf
import numpy as np

(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar10.load_data()

# Normalization
x_train=x_train/255.

x_test=x_test/255.

x_train.shape

x_train = tf.image.resize(x_train, [229,229]) 

x_test = tf.image.resize(x_test, [229,229]) 

y_train = tf.keras.utils.to_categorical(np.array(y_train))
y_test = tf.keras.utils.to_categorical(np.array(y_test))

In [3]:
def Stem(x):
    x=layers.Conv2D(filters=32,kernel_size=(3,3),strides=(2,2),activation='relu')(x)
    x=layers.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),activation='relu')(x)
    x=layers.Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x)

    x1=layers.MaxPooling2D(pool_size=(3,3),strides=(2,2))(x)
    x2=layers.Conv2D(filters=96,kernel_size=(3,3),strides=(2,2),activation='relu')(x)
    x3=tf.keras.layers.Concatenate()([x1, x2])

    x4=layers.Conv2D(filters=64,kernel_size=(1,1),strides=(1,1),activation='relu')(x3)
    x4=layers.Conv2D(filters=96,kernel_size=(3,3),strides=(1,1),activation='relu')(x4)

    x5=layers.Conv2D(filters=64,kernel_size=(1,1),strides=(1,1),activation='relu')(x3)
    x5=layers.Conv2D(filters=64,kernel_size=(7,1),strides=(1,1),padding='same',activation='relu')(x5)
    x5=layers.Conv2D(filters=64,kernel_size=(1,7),strides=(1,1),padding='same',activation='relu')(x5)
    x5=layers.Conv2D(filters=96,kernel_size=(3,3),strides=(1,1),activation='relu')(x5)
    x6=tf.keras.layers.Concatenate()([x4, x5])

    x7=layers.MaxPooling2D(pool_size=(3,3),strides=(2,2))(x6)
    x8=layers.Conv2D(filters=192,kernel_size=(3,3),strides=(2,2),activation='relu')(x6)
    x9=tf.keras.layers.Concatenate()([x7, x8])

    return x9


def Inception_ResNet_Block_A(x):
    x=layers.ReLU()(x)

    x1=layers.Conv2D(filters=32,kernel_size=(1,1),strides=(1,1),activation='relu',padding='same')(x)

    x2=layers.Conv2D(filters=32,kernel_size=(1,1),strides=(1,1),activation='relu',padding='same')(x)
    x2=layers.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(x2)


    x3=layers.Conv2D(filters=32,kernel_size=(1,1),strides=(1,1),activation='relu',padding='same')(x)
    x3=layers.Conv2D(filters=48,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(x3)
    x3=layers.Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(x3)

    x4=tf.keras.layers.Concatenate()([x,x1,x2,x3])
    x4=layers.Conv2D(filters=384,kernel_size=(1,1),strides=(1,1),padding='same')(x4)
    x5=tf.keras.layers.Concatenate()([x,x4])

    x5=layers.ReLU()(x5)
    return x5



def Inception_ResNet_Block_B(x):
    x=layers.ReLU()(x)

    x1=layers.Conv2D(filters=192,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)

    x2=layers.Conv2D(filters=128,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)
    x2=layers.Conv2D(filters=160,kernel_size=(1,7),strides=(1,1),padding='same',activation='relu')(x2)
    x2=layers.Conv2D(filters=192,kernel_size=(7,1),strides=(1,1),padding='same',activation='relu')(x2)
    
    x3=tf.keras.layers.Concatenate()([x1,x2])

    x3=layers.Conv2D(filters=1154,kernel_size=(1,1),strides=(1,1),padding='same')(x3)
    
    x4=tf.keras.layers.Concatenate()([x,x3])

    x4=layers.ReLU()(x4)

    return x4


def Inception_ResNet_Block_C(x):
    x=layers.ReLU()(x)

    x1=layers.Conv2D(filters=192,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)

    x2=layers.Conv2D(filters=192,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)
    x2=layers.Conv2D(filters=224,kernel_size=(1,3),strides=(1,1),padding='same',activation='relu')(x2)
    x2=layers.Conv2D(filters=256,kernel_size=(3,1),strides=(1,1),padding='same',activation='relu')(x2)
    
    x3=tf.keras.layers.Concatenate()([x1,x2])

    x3=layers.Conv2D(filters=2048,kernel_size=(1,1),strides=(1,1),padding='same')(x3)
    
    x4=tf.keras.layers.Concatenate()([x,x3])

    x4=layers.ReLU()(x4)

    return x4


def Reduction_Block_A(input_1,input_2,input_3,input_4,input_5):
    x=tf.keras.layers.Concatenate()([input_1,input_2,input_3,input_4,input_5]) 
    x=layers.MaxPooling2D(pool_size=(3,3),strides=(2,2))(x)

    x1=layers.Conv2D(filters=384,kernel_size=(3,3),strides=(2,2),activation='relu')(x)

    x2=layers.Conv2D(filters=256,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)
    x2=layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x2)
    x2=layers.Conv2D(filters=384,kernel_size=(3,3),strides=(2,2),activation='relu')(x2)

    x3=tf.keras.layers.Concatenate()([x1,x2])
    
    return x3 



def Reduction_Block_B(x):
    x1=layers.MaxPooling2D(pool_size=(3,3),strides=(2,2))(x)

    x2=layers.Conv2D(filters=256,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)
    x2=layers.Conv2D(filters=384,kernel_size=(3,3),strides=(2,2),activation='relu')(x2)
    

    x3=layers.Conv2D(filters=256,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)
    x3=layers.Conv2D(filters=288,kernel_size=(3,3),strides=(2,2),activation='relu')(x3)

    x4=layers.Conv2D(filters=256,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)
    x4=layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x)
    x4=layers.Conv2D(filters=320,kernel_size=(3,3),strides=(2,2),activation='relu')(x)
    

    x5=tf.keras.layers.Concatenate()([x1,x2,x2,x3,x4])
    
    return x5 

In [28]:

input=layers.Input(shape=(229,229,3))
x=Stem(input)
    
x1=Inception_ResNet_Block_A(x)
x2=Inception_ResNet_Block_A(x)
x3=Inception_ResNet_Block_A(x)
x4=Inception_ResNet_Block_A(x)
x5=Inception_ResNet_Block_A(x)
    
x6=Reduction_Block_A(x1,x2,x3,x4,x5)
    
x6=Inception_ResNet_Block_B(x6)#1
x6=Inception_ResNet_Block_B(x6)#2
x6=Inception_ResNet_Block_B(x6)#3
x6=Inception_ResNet_Block_B(x6)#4
x6=Inception_ResNet_Block_B(x6)#5
x6=Inception_ResNet_Block_B(x6)#6
x6=Inception_ResNet_Block_B(x6)#7
x6=Inception_ResNet_Block_B(x6)#8
x6=Inception_ResNet_Block_B(x6)#9
x6=Inception_ResNet_Block_B(x6)#10
    
x6=Reduction_Block_B(x6)
    
x6=Inception_ResNet_Block_C(x6)#1
x6=Inception_ResNet_Block_C(x6)#2
x6=Inception_ResNet_Block_C(x6)#3
x6=Inception_ResNet_Block_C(x6)#4
x6=Inception_ResNet_Block_C(x6)#5
    
x6=layers.GlobalAveragePooling2D()(x6)
x6=layers.Dropout(0.8)(x6)
x6=layers.Dense(units=10,activation='softmax')(x6)
    
model = tf.keras.models.Model(input,x6)

In [29]:
model.build(input_shape=(229, 229, 3))

In [30]:
plot_model(model,to_file='Inception_ResNet_v2.png',show_shapes=True)

In [31]:
model.compile(optimizer='adam',loss='categorical_crossentropy' ,metrics=['acc'])

In [32]:
history=model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=1,batch_size=8,verbose=1)

  10/6250 [..............................] - ETA: 5:31:40 - loss: 2.5465 - acc: 0.0750